In [18]:
import sys, os
sys.path.append(os.path.abspath("../src"))

from factor_calculations import compute_log_returns
from backtest_engine import walk_forward_ls_by_regime
from regime_definitions import compute_market_volatility_regime
from stats_helpers import bootstrap_mean_diff       # bootstrap test

import pandas as pd
import numpy as np

from itertools import product
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Step 1: Import dfs

In [2]:
# Load data
price_df = pd.read_parquet("../data/processed/price_filtered.parquet")       # raw prices for regime calc
momentum_df = pd.read_parquet("../data/processed/momentum.parquet")
returns_df = compute_log_returns(price_df)

In [ ]:
# Step 2: Define Parameter Grid

In [3]:
vol_windows = [10, 21, 63]         # days
test_lengths = [5, 21, 63]         # days
regime_lags = [0, 5, 10]           # days

In [ ]:
# Step 3: Run Grid

In [5]:
results = []

for vw, tl, lag in product(vol_windows, test_lengths, regime_lags):
    # Compute regime using prebuilt function
    regime = compute_market_volatility_regime(price_df, window=vw)

    # Evaluate walk-forward returns in each regime
    stats = {}
    for reg_label in ["low_vol", "high_vol"]:
        res = walk_forward_ls_by_regime(
            factor=momentum_df,
            returns=returns_df,
            regime=regime,
            target_regime=reg_label,
            lag_days=lag,
            train_window=252,
            test_window=tl
        )
        stats[reg_label] = res

    # Bootstrap difference between regimes
    bootstrap_result = bootstrap_mean_diff(
        stats["low_vol"]["returns"],
        stats["high_vol"]["returns"]
    )
    diff_pval = bootstrap_result["p_value"]
    diff_mean = bootstrap_result["mean_diff"]
    

    # Save
    results.append({
        "vol_window": vw,
        "test_len": tl,
        "lag_days": lag,
        "mean_low": stats["low_vol"]["returns"].mean(),
        "mean_high": stats["high_vol"]["returns"].mean(),
        "mean_diff": diff_mean,
        "pval": diff_pval
    })

In [ ]:
# Step 4: Store Outputs

In [6]:
# Convert to DataFrame and save
results_df = pd.DataFrame(results)

In [8]:
results_df.to_parquet("../data/processed/sensitivity_results.parquet")

In [9]:
results_df.to_csv("../data/processed/sensitivity_results.csv")

In [14]:
# Plot

In [16]:
results_df.head()


,vol_window,test_len,lag_days,mean_low,mean_high,mean_diff,pval
0,10,5,0,0.000092,-0.000101,0.000192,0.757
1,10,5,5,0.000155,-0.000511,0.000666,0.527
2,10,5,10,0.000408,-0.000486,0.000894,0.486
3,10,21,0,0.000092,-0.000095,0.000187,0.786
4,10,21,5,0.000155,-0.000511,0.000666,0.501


In [24]:
for lag in results_df['lag_days'].unique():

    pivot = results_df[results_df["lag_days"] == lag].pivot(
        index="vol_window",
        columns="test_len",
        values="mean_diff"
    )

    sns.heatmap(pivot, annot=True, fmt=".5f", cmap="coolwarm")
    plt.title("Mean Return Difference (Low - High Vol), Lag = " + str(lag))
    plt.xlabel("Test Window Length")
    plt.ylabel("Volatility Regime Window")
    plt.tight_layout()
    plt.savefig("../research/plots/heatmap_mean_diff_lag" + str(lag) + ".png")
    plt.clf()  # ← resets the figure

<Figure size 432x288 with 0 Axes>

While momentum showed directionally stronger returns in low-volatility regimes across all grid settings, no configuration produced statistically significant alpha at conventional confidence levels. This suggests regime-aware signals may exist but are not reliably extractable using standard decile-sorting approaches on daily data.